## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-02-23-51-46 +0000,bbc,Google avoids break-up but must share data wit...,https://www.bbc.com/news/articles/cp8zdrenm1zo...
1,2025-09-02-23-43-34 +0000,nyt,"House Oversight Panel Releases 33,000 Pages of...",https://www.nytimes.com/2025/09/02/us/politics...
2,2025-09-02-23-37-07 +0000,bbc,Don't pay a fake parking fine – four ways to p...,https://www.bbc.com/news/articles/cn8438ngpe1o...
3,2025-09-02-23-37-03 +0000,startribune,Trump says he's set to order federal intervent...,https://www.startribune.com/trump-says-he-will...
4,2025-09-02-23-36-33 +0000,cbc,U.S. military kills 11 in strike on alleged dr...,https://www.cbc.ca/news/world/trump-venezuela-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2324/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
27,trump,53
325,china,16
180,new,15
28,he,11
8,house,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
153,2025-09-02-17-48-32 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...,104
198,2025-09-02-14-16-14 +0000,latimes,Trump deployment of military troops to Los Ang...,https://www.latimes.com/california/story/2025-...,103
124,2025-09-02-19-04-50 +0000,nypost,‘Morning Joe’ host Joe Scarborough urges defia...,https://nypost.com/2025/09/02/us-news/joe-scar...,102
319,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...,102
4,2025-09-02-23-36-33 +0000,cbc,U.S. military kills 11 in strike on alleged dr...,https://www.cbc.ca/news/world/trump-venezuela-...,96


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
153,104,2025-09-02-17-48-32 +0000,bbc,Trump's use of National Guard in Los Angeles w...,https://www.bbc.com/news/articles/c5ylyd9lkkqo...
76,50,2025-09-02-21-00-26 +0000,wapo,China to flex military muscle with parade of ‘...,https://www.washingtonpost.com/world/2025/09/0...
289,42,2025-09-02-05-23-02 +0000,bbc,Chris Mason: Starmer faces pressure to deliver...,https://www.bbc.com/news/articles/c8jpj8knvn8o...
29,40,2025-09-02-22-21-30 +0000,nypost,"Zohran Mamdani’s brain trust is full of young,...",https://nypost.com/2025/09/02/us-news/zohran-m...
298,37,2025-09-02-03-43-25 +0000,nypost,Chef killed in electric buggy crash after late...,https://nypost.com/2025/09/01/world-news/chef-...
11,35,2025-09-02-23-16-19 +0000,nypost,Dramatic footage shows US strike that oblitera...,https://nypost.com/2025/09/02/world-news/foota...
319,34,2025-09-02-00-05-47 +0000,nypost,Bessent says Trump admin willing to hit Russia...,https://nypost.com/2025/09/01/world-news/besse...
273,33,2025-09-02-08-23-36 +0000,nypost,"Sudan landslide kills over 1,000 people and wi...",https://nypost.com/2025/09/02/world-news/sudan...
164,33,2025-09-02-16-58-29 +0000,nypost,Trump claims video of items thrown from White ...,https://nypost.com/2025/09/02/us-news/viral-vi...
112,31,2025-09-02-19-58-20 +0000,nypost,EU antitrust regulators delay Google adtech fi...,https://nypost.com/2025/09/02/business/eu-anti...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
